In [1]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', None)        # Let the display adjust to the window

# 2. Set the display width (optional but often helpful)
#    'None' tries to detect terminal width. 
#    A large number (e.g., 1000) ensures no wrapping unless absolutely necessary.
pd.set_option('display.width', 1000) 

In [2]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")
print(f"DOWNLOAD_DIR: {DOWNLOAD_DIR}")
print(f"DEST_DIR: {DEST_DIR}\n")

# Build paths
source_path = Path(DOWNLOAD_DIR) / f'df_OHLCV_{date_str}.parquet'
dest_path = Path(DEST_DIR) / f'{date_str}_df_OHLCV_clean.parquet'

print(f"source_path: {source_path}")
print(f"dest_path: {dest_path}")

date_str: 2025-04-07
DOWNLOAD_DIR: C:\Users\ping\Downloads
DEST_DIR: ..\data

source_path: C:\Users\ping\Downloads\df_OHLCV_2025-04-07.parquet
dest_path: ..\data\2025-04-07_df_OHLCV_clean.parquet


In [3]:
# --- Data Loading & Initial Inspection ---
import pandas as pd

# # Load raw data from pickle file
# df = pd.read_pickle(source_path)

df = pd.read_parquet(source_path, engine='pyarrow')

# Display initial data structure
print("[Raw Data Overview]")
display(df.head())
df.info()

[Raw Data Overview]


Open   High    Low  Close  Adj Close   Volume  Adj Open  Adj High  Adj Low
Symbol Date                                                                                   
SCHC   2025-04-07  32.15  33.41  30.84  32.41      32.41  1831992     32.15     33.41    30.84
       2025-04-04  34.10  34.15  33.04  33.15      33.15   820300     34.10     34.15    33.04
       2025-04-03  35.81  36.03  35.39  35.42      35.42   288400     35.81     36.03    35.39
       2025-04-02  35.64  36.20  35.64  36.08      36.08   586300     35.64     36.20    35.64
       2025-04-01  35.92  36.10  35.74  35.96      35.96   701800     35.92     36.10    35.74

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 388995 entries, ('SCHC', Timestamp('2025-04-07 00:00:00')) to ('WAB', Timestamp('2024-04-08 00:00:00'))
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Open       388995 non-null  float64
 1   High       388995 non-null  float64
 2   Low        388995 non-null  float64
 3   Close      388995 non-null  float64
 4   Adj Close  388995 non-null  float64
 5   Volume     388977 non-null  Int64  
 6   Adj Open   388995 non-null  float64
 7   Adj High   388995 non-null  float64
 8   Adj Low    388995 non-null  float64
dtypes: Int64(1), float64(8)
memory usage: 28.6+ MB


In [4]:
import sys
from pathlib import Path

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))


# --- Data Filtering & Cleaning ---
import utils  # Custom utility functions

# 1. Align dates across all symbols using SPY as reference
df = utils.filter_df_dates_to_reference_symbol(df=df, reference_symbol='SPY')

# 2. Remove symbols with missing data points
df_clean, missing_symbols = utils.filter_symbols_with_missing_values(df)

# Display cleaning results
print("\n[Cleaning Report]")
print(f"Removed {len(missing_symbols)} symbols with missing data: {missing_symbols}")
print("\n[Cleaned Data Structure]")
df_clean.info()

Original number of symbols: 1558
Number of symbols after filtering: 1538
Number of symbols filtered out: 20

First 10 symbols that were filtered out:
['LINE', 'CRWV', 'VG', 'LOAR', 'NMAX', 'LTM', 'RBRK', 'TTAN', 'WAY', 'ULS']

Example of dates for first filtered out symbol:

Dates for LINE:
DatetimeIndex(['2025-04-07', '2025-04-04', '2025-04-03', '2025-04-02', '2025-04-01', '2025-03-31', '2025-03-28', '2025-03-27', '2025-03-26', '2025-03-25',
               ...
               '2024-08-07', '2024-08-06', '2024-08-05', '2024-08-02', '2024-08-01', '2024-07-31', '2024-07-30', '2024-07-29', '2024-07-26', '2024-07-25'], dtype='datetime64[ns]', name='Date', length=176, freq=None)

Filtered DataFrame info:
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 386038 entries, ('SCHC', Timestamp('2025-04-07 00:00:00')) to ('WAB', Timestamp('2024-04-08 00:00:00'))
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Open       386038 n


[Cleaning Report]
Removed 2 symbols with missing data: ['FSEC', 'FER']

[Cleaned Data Structure]
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 385536 entries, ('SCHC', Timestamp('2025-04-07 00:00:00')) to ('WAB', Timestamp('2024-04-08 00:00:00'))
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Open       385536 non-null  float64
 1   High       385536 non-null  float64
 2   Low        385536 non-null  float64
 3   Close      385536 non-null  float64
 4   Adj Close  385536 non-null  float64
 5   Volume     385536 non-null  Int64  
 6   Adj Open   385536 non-null  float64
 7   Adj High   385536 non-null  float64
 8   Adj Low    385536 non-null  float64
dtypes: Int64(1), float64(8)
memory usage: 28.4+ MB


In [5]:
# --- Save Cleaned Data ---
# # Save processed data to pickle file
# df_clean.to_pickle(dest_path)

# Using PyArrow (default, recommended for most cases)
df_clean.to_parquet(dest_path, engine='pyarrow')
print(f"\n[Save Successful] Cleaned data saved to:\n{dest_path}")




[Save Successful] Cleaned data saved to:
..\data\2025-04-07_df_OHLCV_clean.parquet
